In [1]:
import torch
import meshzoo

from torchfem import Shell

## Two simple triangular shell elements

In [2]:
# Material model (plane stress)
E = 1000.0
nu = 0.3
C = (E / (1.0 - nu**2)) * torch.tensor(
    [[1.0, nu, 0.0], [nu, 1.0, 0.0], [0.0, 0.0, 0.5 * (1.0 - nu)]]
)

# Create nodes and element
nodes = torch.tensor(
    [[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [1.0, 1.0, 0.0]]
)
elements = [[1, 2, 0], [3, 2, 1]]


# Constrained displacement at left end
forces = torch.zeros_like(nodes)
displacements = torch.zeros_like(nodes)
displacements[nodes[:, 0] > 0.99, 0] = 1.0
constraints = torch.ones_like(nodes, dtype=bool)

# Thickness
thickness = 0.1 * torch.ones(len(elements))

rectangle = Shell(nodes, elements, forces, displacements, constraints, thickness, C)
rectangle.plot()

Widget(value="<iframe src='http://localhost:50814/index.html?ui=P_0x28c5f3490_1&reconnect=auto' style='width: …

In [3]:
# Solve
u, f = rectangle.solve()
sigma = rectangle.compute_stress(u)
# Plot
rectangle.plot(u, node_property={"u": torch.norm(u, dim=1)})

Widget(value="<iframe src='http://localhost:50814/index.html?ui=P_0x173d159d0_2&reconnect=auto' style='width: …

## Membrane tube

In [4]:
points, cells = meshzoo.tube(length=1.0, radius=0.5, n=10)
nodes = torch.tensor(points, dtype=torch.float32)
top = nodes[:, 2] > 0.49
middle = (nodes[:, 2] < 0.01) & (nodes[:, 2] > -0.01)
bottom = nodes[:, 2] < -0.49
elements = cells.tolist()
forces = torch.zeros_like(nodes)
displacements = torch.zeros_like(nodes)
displacements[top, 2] = 0.1
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[bottom] = True
constraints[middle, 0] = True
constraints[middle, 1] = True
constraints[top] = True
thickness = 0.1 * torch.ones(len(elements))

# Thickness
thickness = 0.1 * torch.ones(len(elements))
tube = Shell(nodes, elements, forces, displacements, constraints, thickness, C)
tube.plot()

Widget(value="<iframe src='http://localhost:50814/index.html?ui=P_0x173db9150_3&reconnect=auto' style='width: …

First stiffness matrix should be (Abaqus reference)
```
  1.13,
 -3.49,  10.75  
  0.00,   0.00,  33.95
  0.00,   0.00,   5.09,   8.48   
  0.00,   0.00, -15.67, -26.12,  80.41
  5.94, -18.28,   0.00,   0.00,   0.00,  31.11 
 -1.13,  3.492,  -5.09,  -8.48,  26.12,  -5.94,  9.62  
  3.49, -10.75,  15.67,  26.12, -80.41,  18.28, -29.62,  91.16
 -5.94,  18.28, -33.95,  -5.09,  15.67, -31.11,  11.03, -33.96, 65.07
 ``````

In [5]:
for i, row in enumerate(tube.k()[0]):
    print(",".join([f"{j:7.2f}" for j in row[0 : i + 1]]))

   1.13
  -3.49,  10.75
  -0.00,   0.00,  33.96
  -0.00,   0.00,   5.09,   8.49
   0.00,  -0.00, -15.68, -26.13,  80.41
   5.94, -18.29,  -0.00,  -0.00,   0.00,  31.12
  -1.13,   3.49,  -5.09,  -8.49,  26.13,  -5.94,   9.62
   3.49, -10.75,  15.68,  26.13, -80.41,  18.29, -29.62,  91.16
  -5.94,  18.29, -33.96,  -5.09,  15.68, -31.12,  11.04, -33.97,  65.07


In [6]:
# Solve
u, f = tube.solve()
tube.plot(u, node_property={"u_z": u[:, 2]})

Widget(value="<iframe src='http://localhost:50814/index.html?ui=P_0x16b555e50_4&reconnect=auto' style='width: …

In [7]:
sigma = tube.compute_stress(u)  # Plot
tube.plot(u, element_property={"S_zz": sigma[:, 2, 2]})

Widget(value="<iframe src='http://localhost:50814/index.html?ui=P_0x173dc0110_5&reconnect=auto' style='width: …